# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import numpy as np
import tensorflow as tf

import time
import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [8]:
from tqdm import tqdm
from os.path import isdir, isfile
from urllib.request import urlretrieve
import zipfile

dataset_filename = 'text8.zip'
dataset_folder = 'data'
dataset_name = 'Text 8 data'

class DLProgress(tqdm):
    last_block = 0
    
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
        
if not isfile(dataset_filename):
    with DLProgress() as pbar:
        urlretrieve('http://mattmahoney.net/dc/text8.zip', dataset_filename, pbar.hook)
        
if isdir(dataset_folder):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder)
        
with open('data/text8') as f:
    text = f.read()

In [12]:
text[:100]

' anarchism originated as a term of abuse first used against early working class radicals including t'

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [13]:
words = utils.preprocess(text)

In [15]:
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [16]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [46]:
from collections import Counter

word_counts = Counter(words)


In [73]:
word_counts.get('the')

1061396

In [65]:
# print(word_counts.most_common()[:10])
for i in range(5):
    print(word_counts.popitem())
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(sorted_vocab[:10])

('wigman', 6)
('baybears', 6)
('nrms', 9)
('matzo', 6)
('konservatorium', 7)
['the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two']


In [79]:
for i in range(50):
    print(sorted_vocab[i], word_counts.get(sorted_vocab[i]))

the 1061396
of 593677
and 416629
one 411764
in 372201
a 325873
to 316376
zero 264975
nine 250430
two 192644
is 183153
as 131815
eight 125285
for 118445
s 116710
five 115789
three 114775
was 112807
by 111831
that 109510
four 108182
six 102145
seven 99683
with 95603
on 91250
are 76527
it 73334
from 72871
or 68945
his 62603
an 61925
be 61281
this 58832
which 54788
at 54576
he 53573
also 44358
not 44033
have 39712
were 39086
has 37866
but 35358
other 32433
their 31523
its 29567
first 28810
they 28553
some 28161
had 28100
all 26229


In [68]:
for i,w in enumerate(sorted_vocab[:10]):
    print(i,w)

0 the
1 of
2 and
3 one
4 in
5 a
6 to
7 zero
8 nine
9 two


In [69]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [70]:
vocab_to_int['or']

28

In [71]:
int_to_vocab[28]

'or'

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [97]:
# from collections import Counter
# import random

# threshold = 1e-5
# word_counts = Counter(int_words)
# total_count = len(int_words)
# freqs = {word: count/total_count for word, count in word_counts.items()}
# p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
# # train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]
# p_drop.popitem()

In [90]:
total

16680599

In [95]:
t = 0.00001
t

1e-05

In [98]:
## Your code here

word_counts = Counter(int_words)
total = len(words)
# t = 1e-5
t = 166
# freqs = {word: word_counts.get(word)/total for word in word_counts}
# freqs = {word: count/total for word, count in word_counts.items()}
pw = {word:(1-np.sqrt(t/word_counts.get(word))) for word in word_counts}
# pw = {word:(1-np.sqrt(t/freqs[word])) for word in word_counts}
# pw.popitem()
train_words = [word for word in int_words if pw[word] < random.random()]

In [99]:
train_words[:10]

[5233, 3133, 10571, 27349, 854, 15067, 58112, 854, 3580, 10712]

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [109]:
random.randrange(2, 5)

4

In [116]:
def get_target(words, idx, window_size=5):
    R = random.randrange(1, window_size+1)
#     R = np.random.randint(1, window_size+1)
    start = idx-R if (idx-R > 0) else 0
    stop = idx+R
    return list(set(words[start:idx-1]+words[idx+1:stop]))

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [117]:
def get_batches(words, batch_size, window_size):
    n_batches = len(words)//batch_size
    words = words[:(n_batches*batch_size)]
    
    for i in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[i:i+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [140]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, (None))
    labels = tf.placeholder(tf.int32, (None, None))

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [141]:
inputs.shape

TensorShape(None)

In [142]:
n_vocab = len(int_to_vocab)
n_embeddings = 200
with train_graph.as_default():
    embedding = tf.Variable(tf.random_normal((n_vocab, n_embeddings), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [143]:
n_sampled = 100

with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embeddings),stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [144]:
with train_graph.as_default():
    valid_window = 100
    valid_size = 16
    
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, np.array(random.sample(range(1000, 1000+valid_window), valid_size//2)))
    
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    norm = tf.sqrt(tf.reduce_mean(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding/norm
    
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [145]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

mkdir: checkpoints: File exists


## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [146]:
labels.shape

TensorShape([Dimension(None), Dimension(None)])

In [ ]:
epochs = 2 
batch_size = 1000
window_size = 10

total_iteration = (len(train_words)//batch_size) * epochs

with train_graph.as_default():
    saver = tf.train.Saver()
    
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.initialize_all_variables())
    loss = 0
    iteration = 1
    
    for e in range(epochs):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        
        for x,y in batches:
            train_loss, _ = sess.run([cost, optimizer], feed_dict={inputs:x, labels:np.array(y)[:,None]})
            loss += train_loss
            
            if iteration%500 == 0:
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                     "Iteration {}/{}".format(iteration, total_iteration),
                     "Loss {:.4f}".format(loss/100),
                     "Time {:.4f} sec/batch".format((end-start)/100))

            if iteration%2000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k=8
                    nearest = (sim[i,:]).argsort()[0:top_k+1]
                    log = 'Nearest to %s: ' % valid_word
                    for word_int in nearest:
                        log = '%s %s' %(log, int_to_vocab[word_int])
                    print(log)
            
            iteration += 1 
            
    saver.save('checkpoints/text8.ckpt')
    emb_mat  = sess.run(normalized_embedding)

Epoch 0/2 Iteration 500/9240 Loss 40.7615 Time 1.4850 sec/batch
Epoch 0/2 Iteration 1000/9240 Loss 100.9408 Time 2.9523 sec/batch
Epoch 0/2 Iteration 1500/9240 Loss 169.3391 Time 4.5217 sec/batch
Epoch 0/2 Iteration 2000/9240 Loss 241.6681 Time 6.4395 sec/batch
Nearest to from this article be gilbert coordination invested for snacks satellite
Nearest to up make used other first actually high zero be tract
Nearest to often of is from large as used both when also
Nearest to i be ground zero of from other on scotland while
Nearest to with these wounds were from five breaking supplementary younger clayton
Nearest to years for all of have made at their including popular
Nearest to such direction particular second presence j event both vogt newport
Nearest to than the this from is also of other has soon
Nearest to derived on this be from with an see significant town
Nearest to applied from on be was but for sun is used
Nearest to road of on is the more used actually four five
Nearest to free

Restore the trained network if you need to:

In [150]:
with train_graph.as_default():
    saver = tf.train.Saver()
    
with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

ValueError: Can't load save_path when it is None.

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 2
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(14, 14)
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx,:], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)